In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib

from numba import jit

import sys
sys.path.append('..')
from lib import *
from lib.maxent import *
plt.style.use('../peptidome.mplstyle')

from common import model_hierarchy, labels

In [17]:
datasets = ['train', 'test', 'flat', 'independent']
datasets.extend(model_hierarchy)
sample_matrices = {}
for dataset in datasets:
    sample_matrices[dataset] =  load_matrix('data/%s_matrix_L9.csv.gz' % dataset)[::9, :]

In [18]:
nsample = 2e8
L = sample_matrices['test'].shape[1]
hists_dict = {}
for dataset in datasets:
    hists_dict[dataset] = pairwise_distances_jit(sample_matrices[dataset], N=nsample)

In [19]:
hists_dict['train_test'] = pairwise_distances_jit(sample_matrices['train'],
                                                  data2=sample_matrices['test'],
                                                  N=nsample)

In [20]:
bins = np.arange(L+1)
fig, ax = plt.subplots()
datasets = ['flat', 'test', 'train_test']
datasets.extend(model_hierarchy)
for dataset in datasets"
    ax.step(bins, hists_dict[dataset], label=dataset, where='mid')
ax.set_yscale('log')
ax.set_xticks(bins)
ax.set_xlim(bins[0], bins[-1])
ax.legend(loc='upper left');

SyntaxError: EOL while scanning string literal (<ipython-input-20-e193c107a343>, line 5)

In [9]:
arr = np.load('../aafreqpca/data/data.npz')
aa_human = arr['human']
ps = aa_human
blocklength = sample_matrices['test'].shape[0]//len(ps)
sampless = []
for p in ps:
    samples = np.random.choice(np.arange(0, 20, 1),
                               size=(blocklength, sample_matrices['test'].shape[1]),
                               p=p)
    sampless.append(samples)
sample_composition = np.concatenate(sampless)

In [10]:
hists_dict['composition'] = pairwise_distances_jit(sample_composition, N=nsample)

In [ ]:
fig, ax = plt.subplots()
datasets = ['flat', 'independent']
datasets.extend(model_hierarchy)
for dataset in datasets:
    ax.errorbar(bins, (hists_dict[dataset])/(hists_dict['train_test']),
                hists_dict[dataset]**.5/hists_dict['train_test'],
                label=labels[dataset])
ax.set_xticks(bins)
ax.axhline(1.0, c='k')
ax.set_ylim(0.0, 1.1)
ax.legend(loc='lower right')
ax.set_xlabel('Hamming distance')
ax.set_ylabel('Fraction relative to train | test')
fig.tight_layout()

In [ ]:
pairwise_hist = {}
for dataset in datasets:
    pairwise_hist[dataset] = pairwise_distances_jit(sample_matrices[dataset],
                                   data2=sample_matrices['test'],
                                   N=nsample)

In [ ]:
fig, ax = plt.subplots()
ax.step(bins, hists_dict['test'], where='mid', label='data|data')
for dataset in datasets:
    ax.step(bins, pairwise_hist[dataset], where='mid', label=dataset + '|data')
ax.set_yscale('log')
ax.set_xticks(bins);
ax.legend(loc='upper left')
ax.set_xlabel('Hamming distance')
ax.set_ylabel('Count');

In [ ]:
fig, ax = plt.subplots()
for dataset in datasets:
    ax.errorbar(bins, pairwise_hist[dataset]/hists_dict['train_test'],
                pairwise_hist[dataset]**.5/hists_dict['train_test'],
                label=labels[dataset] + ' | test')
ax.set_xticks(bins)
ax.legend(loc='lower right')
ax.axhline(1.0, c='k')
ax.set_xlabel('Hamming distance')
ax.set_ylabel('Fraction relative to train | test')
fig.tight_layout()
fig.savefig('../../paper/images/hammingdist.pdf')